In [1]:
import snowflake.connector
from snowflake.connector.cursor import SnowflakeCursor
import json
import pandas as pd
import numpy as np

con = snowflake.connector.connect(user='dariush.hale@scale.com',
                         account='pxa65918',
                         authenticator='externalbrowser',
                         warehouse='COMPUTE_WH',
                         database='SCALE_CRAWLER',
                         role='GENERAL_RO')
cs = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


DatabaseError: 250001 (08001): Failed to connect to DB: pxa65918.snowflakecomputing.com:443. The user you were trying to authenticate as differs from the user currently logged in at the IDP.

In [2]:
sql = f'''
select
pvid p_vid,
processed_attributes attributes
from
outputvairants
order by random()
limit 250000
'''
cs.execute(sql)
df = cs.fetch_pandas_all()
df.head(2)

NameError: name 'cs' is not defined

In [48]:
import pandas as pd
import json

df1 = df.copy()

def category(attributes):
    try:
        cat = json.loads(attributes)['fb_product_category']
        return cat
    except Exception:
        return None
    
df1['CATEGORY'] = df1['ATTRIBUTES'].apply(category)

mask = df1['CATEGORY'].isnull()
df1 = df1[~mask]


###attribute AI###
# att = ['capacity', 'concern', 'dimensions', 'effect', 'form', 'health_concern', 'is_water_resistant', 'material', 'model', 'occasion', 'origin_location', 'product_form', 'product_width', 'product_length', 'product_height', 'product_volume', 'product_weight', 'shape', 'sub_brand', 'target_area', 'material', 'color', 'age_group', 'purchase_logistics', 'ingredients', 'instructions', 'condition', 'body_part', 'fashion_accessory_features', 'activity', 'cuff_style', 'jean_wash', 'model_height', 'model_info', 'occasion', 'season', 'fit_type', 'style', 'neckline', 'activewear_pant_style', 'collar_style', 'coat_jacket_fit', 'coat_jacket_style', 'denim_features', 'dress_length_style', 'pant_length_style', 'pant_leg_style', 'pants_features', 'pattern', 'shorts_features', 'skirt_style', 'sleeve_length_style', 't_shirt_style', 'top_length_style', 'top_t_shirt_features', 'top_t_shirt_style', 'heel_height', 'heel_height_style', 'waist_rise', 'fabric_care_instructions', 'sleeve_length', 'sleeve_style', 'dress_style', 'hem_style', 'inseam', 'jeans_style', 'pant_closure_style', 'pant_fit', 'pant_style', 'shirt_fit', 'shirt_style', 'skirt_length', 'sweater_style', 'shoe_style', 'ring_style', 'gemstone', 'gemstone_scale', 'necklace_style', 'charm_pendant_style', 'watch_band_material', 'watch_style', 'watch_type', 'earring_style', 'plating_material', 'bracelet_style', 'glasses_frame_style', 'waist_style', 'watch_band_length', 'watch_case_diameter', 'hair_color_shade', 'hair_type', 'hat_style', 'scent', 'skin_care_concern', 'skin_type', 'spf_value', 'arm_style', 'audio_features', 'bed_type', 'christmas_tree_features', 'clock_features', 'decor_style', 'finish', 'frame_size', 'home_fragrance_features', 'indoor-outdoor_use', 'is_solid_wood', 'light_bulb_type', 'light_bulb_wattage', 'chain_length', 'gemstone_clarity', 'gemstone_creation_method', 'gemstone_cut', 'jewelry_features', 'metal_stamp-purity', 'ring_size', 'boot_shaft_height', 'boot_style', 'heel_style', 'shoe_closure', 'toe_style']
# df1['ATTRIBUTES'] = df1['ATTRIBUTES'].apply(lambda x: list(set([x for x,k in json.loads(x).items() if x in att])))
#################

exclude = [
    'availability','title','breadcrumbs','options','link','description','bullets','image_links','videos','currency','brand','brand',
    'description_structured','sku','key_value_pairs','key_value_pairs','fb_product_category','variant_matchable_id','item_group_id',
    'job_id','id','real_price','parent_website_url','purchase_logistics','higher-price','product_id','fabric_care_instructions','higher_price',
    'size','color_family','material_normalized','secondary_color_normalized','toaster_toaster_oven_type','unique_id','display_color'
]

df1['ATTRIBUTES'] = df1['ATTRIBUTES'].apply(lambda x: list(set([x for x,k in json.loads(x).items() if x not in exclude])))

df1 = df1.explode('ATTRIBUTES')

#replacing / mergin attributes
mask = (df1['ATTRIBUTES'].str.contains('features')) | (df1['ATTRIBUTES'].str.contains('size_chart'))
df1 = df1[~mask]
mask = df1['ATTRIBUTES'].str.contains('instructions')
df1 = df1[~mask]

df1['ATTRIBUTES'] = df1['ATTRIBUTES'].replace('activewear_pant_style','pant_style')
df1['ATTRIBUTES'] = df1['ATTRIBUTES'].replace('age','age_group')
df1['ATTRIBUTES'] = df1['ATTRIBUTES'].replace('heel_style','heel_height_style')
df1['ATTRIBUTES'] = df1['ATTRIBUTES'].replace('ingredient_composition','ingredients')
df1['ATTRIBUTES'] = df1['ATTRIBUTES'].replace('origin_country','origin_location')
df1['ATTRIBUTES'] = df1['ATTRIBUTES'].replace('top_t_shirt_style','t_shirt_style')

df1 = df1.groupby(['CATEGORY','ATTRIBUTES'],as_index=False).count().rename(columns={'P_VID':'T_CATEGORY'})

df1 = df1.pivot(index='CATEGORY', columns='ATTRIBUTES', values='T_CATEGORY').reset_index()

col_list= df1.iloc[:,1:].columns
df1['TOTAL'] = df1[col_list].sum(axis=1)

df1 = df1.sort_values('TOTAL',ascending=False)

df1

,P_VID,ATTRIBUTES,CATEGORY
0,superiorsportsinvestments.com! 1251,product_type,Antiques & Collectibles > Sports Memorabilia >...
0,superiorsportsinvestments.com! 1251,sub_brand,Antiques & Collectibles > Sports Memorabilia >...
3,www.namebrandwigs.com!6707608068,color_normalized,Health & Beauty > Beauty > Hair Care & Styling...
3,www.namebrandwigs.com!6707608068,dimensions,Health & Beauty > Beauty > Hair Care & Styling...
3,www.namebrandwigs.com!6707608068,hair_color_shade,Health & Beauty > Beauty > Hair Care & Styling...
...,...,...,...
249994,jane.com!221202412212024722120262,color_normalized,Clothing & Accessories > Women > Women's Cloth...
249994,jane.com!221202412212024722120262,material,Clothing & Accessories > Women > Women's Cloth...
249994,jane.com!221202412212024722120262,product_type,Clothing & Accessories > Women > Women's Cloth...
249994,jane.com!221202412212024722120262,gender,Clothing & Accessories > Women > Women's Cloth...


In [46]:
df1.to_csv('vishal_250k_clean.csv',index=False)

In [ ]:
rank = {}

test = df1.rank(ascending=False, method='first', axis=1)

import re
re1 = re.compile(r'.*instructions.*')
list(filter(re1.match, df.columns))


remove = ['fb_pixel_content_id','free_shipping','food_processor_functions','unique_id','care_instructions']
remo = remove + list(filter(re1.match, df.columns))


test = test[[c for c in test.columns if c not in remo]]

for n,row in test.iterrows():

    fn = {col:row[col] for col in test.columns[:-1] if math.isnan(row[col])==False}
    
    rank[df.iloc[n]['CATEGORY']] = sorted(fn.items(), key=lambda x:x[1])[:15]

pd.DataFrame({'cat':rank.keys(),'att':{k:[a[0] for a in v] for k,v in rank.items()}.values()}).to_csv('rank.csv',index=False)
    